# sparse distance matrix test

Most entries in the distance matrix for the RGG are too large to be interesting and we filter them out when computing statistics, e.g. when computing the connectivity threshold the maximum radius we consider is $O( (log(n) / n)^{1/d} )$.

If we only store the distances between vertices $x,y$ with $\| x - y \| \leq R_{\mathrm{max}}$ where $n R_{\mathrm{max}}^d = O(\log n)$ (a sensible upper bound for most statistics), then the amount of memory we need is $O(n \log n )$ instead of $\Theta(n^2)$, a significant improvement which should let us use RGGs with more than a few thousand vertices.

In [ ]:
import numpy as np
from scipy.spatial import KDTree
from sys import getsizeof
import timeit

rng = np.random.default_rng()

In [ ]:
def full_matrix_experiment(n,d=2):
    # generates the (full) distance matrix for an RGG with n points in [0,1]^d,
    # and returns the amount of memory it uses.
    V = rng.uniform(size=(n,d))
    dm =  np.linalg.norm(V[:,None,:] - V[None,:,:],axis=-1)
    return dm.size

def sparse_matrix_experiment(n,r_max,d=2):
    V = rng.uniform(size=(n,d))
    tree = KDTree(V)
    sdm = tree.sparse_distance_matrix(tree, r_max)
    return sdm.count_nonzero()

Let's check both the size of the resulting matrices, and how long it takes to generate them:

In [ ]:
n = 5000
d = 3
r = 4*(np.log(n)/n)**(1/d) # Quite a bit above the coverage threshold for the square, so the test isn't too biased in favour of the sparse matrix
reps = 5

print(f'Number of elements in the sparse distance matrix with radius {r:.3f}: {sparse_matrix_experiment(n,r,d)}')
print(f'Number of elements in the usual distance matrix:                    {full_matrix_experiment(n,d)}\n')

print(f'Time taken to generate {reps} sparse distance matrices (in seconds):')
print(sum(timeit.repeat(f'sparse_matrix_experiment({n},{r},{d})',repeat=reps,globals=globals(),number=1)))
print(f'Time taken to generate {reps} usual distance matrices (in seconds):')
print(sum(timeit.repeat(f'full_matrix_experiment({n},{d})',repeat=reps,globals=globals(),number=1)))

Number of elements in the sparse distance matrix with radius 0.478: 6178320
Number of elements in the usual distance matrix:                    25000000

Time taken to generate 5 sparse distance matrices (in seconds):


The results are interesting: I think they provide a good case for using sparse distance matrices. Maybe I'll do a few more experiments by trying to calculate the number of components using a sparse distance matrix: this may or may not be quicker.

Generating a usual distance matrix is a bit quicker when $r = 4 ( \frac{\log n}{n} )^{1/d}$, but this is generally much larger than the coverage threshold. If $r = 2 (\frac{\log n}{n})^{1/d}$, then generating a sparse distance matrix seems faster (it's also possible to just convert a 2d numpy array straight into a scipy.sparse.dok_array type sparse array, so maybe we can do without the KDTree step. But I think for very large $n$ the KDTree will be faster than usual linear algebra).

A sparse distance matrix contains many fewer elements than the usual distance matrix. For every non-zero element it does also store the indices, so maybe you should triple the number of elements to get a comparable storage space, but even so it's much smaller than the full distance matrix.

In [ ]:
n = 10000
r = 2*(np.log(n)/n)**(1/2)
sparse_matrix_experiment(n,r,d=2)

1098822

In particular it's possible to compute sparse distance matrices for more points; my computer runs out of memory when trying to compute the full distance matrix for more than around 6,000 points.